<a href="https://colab.research.google.com/github/Calafold/anireveal-engine/blob/main/anireveal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proof of Concept for AI-powered Anime Guessing game



## Install and import relevant packages

In [ ]:
# Installs
%pip install supabase
%pip install google-generativeai

# Imports
from google.colab import userdata
from supabase import create_client, Client
import requests
import time
import json
import google.generativeai as genai

## Connect to database and setup Gemini API



In [11]:
# Connection to supabase
url: str = userdata.get("SUPABASE_URL")
key: str = userdata.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

# Configure the Gemini API with your API key
genai.configure(api_key=userdata.get("GOOGLE_GEMINI_API_KEY"))
model = genai.GenerativeModel('gemini-1.5-flash')

## Populate the database with Anime list of top 100

In [ ]:
def populate_anime_table():
    """
    Fetches the top 100 anime from the Jikan API and populates the
    'anime' table in the Supabase database using upsert.
    """
    anime_list = []

    # Jikan API returns 25 results per page. Loop through 4 pages for the top 100.
    for page in range(1, 5):
        try:
            response = requests.get(f"https://api.jikan.moe/v4/top/anime?page={page}")
            response.raise_for_status()  # Raise an exception for bad status codes
            data = response.json()
            for anime in data['data']:
                anime_list.append({
                    "id": anime['mal_id'],
                    "title": anime['title'],
                    "image_url": anime['images']['jpg']['image_url']
                })
            print(f"Fetched page {page} of top anime.")
            time.sleep(1) # Respect API rate limits

        except requests.exceptions.RequestException as e:
            print(f"Error fetching top anime from page {page}: {e}")
            break

    if anime_list:
        try:
            supabase.table("anime").upsert(anime_list, on_conflict="id").execute()
            print("Successfully populated the 'anime' table with the top 100 entries.")
        except Exception as e:
            print(f"Error upserting anime data into Supabase: {e}")
    else:
        print("No anime data to insert.")


populate_anime_table()

## Populate the database with the list of characters for each anime

In [ ]:
def populate_characters_table():
    """
    Fetches characters for all anime in the database and populates the
    'characters' table using upsert.
    """
    anime_ids_response = supabase.table("anime").select("id").execute()
    anime_ids = [a['id'] for a in anime_ids_response.data]

    for anime_id in anime_ids:
        try:
            response = requests.get(f"https://api.jikan.moe/v4/anime/{anime_id}/characters")
            response.raise_for_status()
            characters_data = response.json()['data']

            characters_to_insert = []
            for char_entry in characters_data:
                # Filter for main characters with an image
                if char_entry['role'] == 'Main' and 'jpg' in char_entry['character']['images']:
                    character_info = char_entry['character']
                    characters_to_insert.append({
                        "id": character_info['mal_id'],
                        "name": character_info['name'],
                        "anime_id": anime_id,
                        "image_url": character_info['images']['jpg']['image_url']
                    })

            if characters_to_insert:
                supabase.table("characters").upsert(characters_to_insert, on_conflict="id").execute()
                print(f"Upserted {len(characters_to_insert)} characters for anime ID {anime_id}")

        except requests.exceptions.RequestException as e:
            print(f"Error fetching/upserting characters for anime ID {anime_id}: {e}")

        time.sleep(1) # Respect API rate limits


populate_characters_table()

## Generate clues using ai and populate database with clues

In [ ]:
def generate_and_populate_clues():
    """
    Generates AI clues for each character and populates the 'clues' table using upsert.
    """
    characters_response = supabase.table("characters").select("id", "name", "anime_id").execute()
    characters = characters_response.data
    print("Characters fetched.")

    anime_titles_response = supabase.table("anime").select("id", "title").execute()
    anime_titles = {a['id']: a['title'] for a in anime_titles_response.data}
    print("Anime titles fetched.")

    for character in characters:
        character_id = character['id']
        character_name = character['name']
        anime_id = character['anime_id']
        anime_title = anime_titles.get(anime_id, "Unknown Anime")

        prompt = f"""Generate 15 unique and distinct clues for the anime character named {character_name} from the anime {anime_title}.

        The clues should be in increasing order of difficulty from 1 (easiest) to 5 (hardest). Provide 3 different clues for each difficulty level.

        Difficulty levels:
        - **Level 1 (Beginner)**: Very direct, common knowledge.
        - **Level 2 (Novice)**: Common, but requires a bit more thought.
        - **Level 3 (Intermediate)**: Relates to a major plot point or personality trait.
        - **Level 4 (Advanced)**: Specific, related to an obscure power or a less-known fact.
        - **Level 5 (Expert)**: Extremely difficult, may relate to their voice actor, a minor character they interacted with, or a subtle design detail.

        Format the output as a JSON object with a single key 'clues'. The value for 'clues' should be an array of objects. Each object in the array must have two keys: 'difficulty' (an integer from 1 to 5) and 'text' (the clue as a string).

        Example Output:
        {{
            "clues": [
                {{
                    "difficulty": 1,
                    "text": "My hair is yellow and spiky and I am a ninja."
                }},
                {{
                    "difficulty": 1,
                    "text": "I am the hero from the anime, Naruto."
                }},
                {{
                    "difficulty": 1,
                    "text": "My catchphrase is 'Believe it!'"
                }},
                {{
                    "difficulty": 2,
                    "text": "I am part of Team 7 with Sakura Haruno and Sasuke Uchiha."
                }},
                {{
                    "difficulty": 2,
                    "text": "I was taught my signature move, the Rasengan, by Jiraiya."
                }},
                {{
                    "difficulty": 2,
                    "text": "I'm the main character in the series about a ninja village."
                }}
            ]
        }}

        Your response must ONLY be the JSON object. Do not include any other text or explanation outside of the JSON."""

        try:
            start_api_call_time = time.time()
            # Increase the timeout for the API call
            response = model.generate_content(prompt, request_options={'timeout': 1200}) # Increased timeout to 1200 seconds
            end_api_call_time = time.time()
            api_call_duration = end_api_call_time - start_api_call_time
            print(f"API call for {character_name} took {api_call_duration:.2f} seconds.")


            clues_json_str = response.text.strip().replace("```json\n", "").replace("\n```", "")
            clues_data = json.loads(clues_json_str)

            clues_to_insert = []
            for clue in clues_data['clues']:
                clues_to_insert.append({
                    "character_id": character_id,
                    "clue_text": clue["text"],
                    "difficulty_level": clue["difficulty"]
                })

            start_db_upsert_time = time.time()
            supabase.table("clues").upsert(clues_to_insert, on_conflict="character_id, difficulty_level").execute()
            end_db_upsert_time = time.time()
            db_upsert_duration = end_db_upsert_time - start_db_upsert_time
            print(f"Database upsert for {character_name} took {db_upsert_duration:.2f} seconds.")

        except Exception as e:
            print(f"Error processing {character_name}: {e}")

        time.sleep(1)

start_time = time.time()
generate_and_populate_clues()
end_time = time.time()
total_duration = end_time - start_time

print(f"Total execution time: {total_duration:.2f} seconds.")